# Alpha/beta hydrolase protein 2090

- Predicted to be an alpha/beta hydrolase.
- Contains Pfam domain [DUF900 (PF05990)](https://www.ebi.ac.uk/interpro/entry/pfam/PF05990/) (positions 61-236)
- From organism _Haloferax sp. s5a-1_, isolated from a saltern in Margherita di Savoia, Italy ([Atanasova et al., 2013](https://doi.org/10.1002/mbo3.115)). 
- Sequenced & assembled by Chahrazad Taissir.

## Protein sequence

```fasta
>gnl|extdb|pgaptmp_002090_1 alpha/beta hydrolase [Haloferax sp. s5a-1] 
MASRRRFLKTTAATFAGLTVFGATSGAAASTPYISTRDHFDDDANLTSGHTARGYDTSGDVPVVDSGSTS
EIFVFAHGWDKNSDNPEQDALEKIAKADTKLTEAGYDCEVVGYTWDSDKGDGWEFGWFEAQEIAQKNGRK
LAQFALDVKRASPGTTVRFTSHSLGAQVIFSALRTLDSRSAWTDSGYTIETMHPFGAATDNEVPGKEEGR
DTYEAIQESAGHVYNYYNAADDVLQWVYNTIEFDQALGETGLEGGDTPAGNYTDRDVESQVGDDHGNYLD
TIADDIVGDI
```

## Homology search

Similar proteins sequences are searched with software [`MMseqs2`](https://github.com/soedinglab/MMseqs2) in two databases: [GTDB(release 214)](https://gtdb.ecogenomic.org/) & [UniProtKB (release 2023_05)](https://www.uniprot.org/help/uniprotkb).

The search procedure works as follows:

1. Search single protein sequence `pgaptmp_002090_1` (alias `2090`) in GTDB & UniProtKB.
2. Map GTDB hits to UniprotKB.
3. Map GTDB hits to our own internal DB, `db_proka`, a phylogenetically balanced subset of GTDB release 214 ([Strock et al., 2024](https://doi.org/10.1101/2024.09.18.613068)).
4. Merge datasets into one.
5. Predict [Pfam (release 37)](https://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam37.0/) domains with [`hmmer`](http://hmmer.org/).

Steps 1, 2 & 3 are implemented in script [`src/homology_search/hydrolase_2090_search.sh`](src/homology_search/hydrolase_2090_search.sh), running on Imperial's HPC.

Step 4 is implemented in this notebook.

Step 5 is implemented in script [`src/homology_search/predict_pfam.sh`](src/homology_search/predict_pfam.sh).

In [202]:
import os
from pathlib import Path
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.SearchIO.HmmerIO.hmmer3_domtab import Hmmer3DomtabHmmqueryParser
from intervaltree import IntervalTree

cwd = os.getcwd()
if cwd.endswith('notebook'):
    os.chdir('..')
    cwd = os.getcwd()

In [203]:
sns.set_palette('colorblind')
sns.set_style('whitegrid')
sns.set_context('paper', font_scale=1.8)
plt.rcParams['font.family'] = 'Helvetica'

palette = sns.color_palette().as_hex()


data_folder = Path('./data')
assert data_folder.is_dir()

## Process homology search results

### Parse GTDB hits

In [204]:
gtdb_hits = pd.read_csv(data_folder / 'hydrolase_search' / 'gtdb.tsv', sep='\t')
gtdb_hits['domain'] = gtdb_hits['taxlineage'].apply(lambda t: t.split(';')[0].replace('d_', ''))
gtdb_hits['gtdb_phylum'] = gtdb_hits['taxlineage'].apply(lambda t: t.split(';')[1].replace('p_', ''))
gtdb_hits['gtdb_class'] = gtdb_hits['taxlineage'].apply(lambda t: t.split(';')[2].replace('c_', ''))
gtdb_hits['gtdb_order'] = gtdb_hits['taxlineage'].apply(lambda t: t.split(';')[3].replace('o_', ''))
gtdb_hits['gtdb_family'] = gtdb_hits['taxlineage'].apply(lambda t: t.split(';')[4].replace('f_', ''))
gtdb_hits['gtdb_genus'] = gtdb_hits['taxlineage'].apply(lambda t: t.split(';')[5].replace('g_', ''))
gtdb_hits['gtdb_species'] = gtdb_hits['taxlineage'].apply(lambda t: t.split(';')[6].replace('s_', ''))
gtdb_hits.head()

,query,target,evalue,bits,tstart,tend,taxlineage,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species
0,pgaptmp_002090_1,NZ_AOLI01000034.1_29,7.391000e-185,581,1,290,d_Archaea;p_Halobacteriota;c_Halobacteria;o_Ha...,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax larsenii
1,pgaptmp_002090_1,NZ_AOLK01000012.1_176,5.303000e-175,552,1,289,d_Archaea;p_Halobacteriota;c_Halobacteria;o_Ha...,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax elongans
2,pgaptmp_002090_1,NZ_AOLN01000018.1_554,2.454000e-141,455,1,290,d_Archaea;p_Halobacteriota;c_Halobacteria;o_Ha...,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax mucosum
3,pgaptmp_002090_1,NC_017941.2_785,3.791000e-139,449,1,290,d_Archaea;p_Halobacteriota;c_Halobacteria;o_Ha...,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax mediterranei
4,pgaptmp_002090_1,NZ_AOLJ01000011.1_106,5.312000e-136,440,1,290,d_Archaea;p_Halobacteriota;c_Halobacteria;o_Ha...,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax gibbonsii


In [205]:
n_archaea = len(gtdb_hits[gtdb_hits['domain'] == 'Archaea'])
n_bacteria = len(gtdb_hits[gtdb_hits['domain'] == 'Bacteria'])

print(f'Number of archaea: {n_archaea:,}')
print(f'Number of bacteria: {n_bacteria:,}')

Number of archaea: 309
Number of bacteria: 19


Distribution of archaeal phyla:

In [206]:
gtdb_hits[gtdb_hits['domain'] == 'Archaea']['gtdb_phylum'].value_counts()

gtdb_phylum
Halobacteriota       160
Thermoproteota       148
Nanohaloarchaeota      1
Name: count, dtype: int64

Distribution of bacterial phyla:

In [207]:
gtdb_hits[gtdb_hits['domain'] == 'Bacteria']['gtdb_phylum'].value_counts()

gtdb_phylum
Chlamydiota          5
4484-113             3
Pseudomonadota       3
Actinomycetota       3
Verrucomicrobiota    2
Bacteroidota         1
Planctomycetota      1
Gemmatimonadota      1
Name: count, dtype: int64

### Map to `db_proka`

Our database `db_proka` is a subset of GTDB, therefore, we'll keep exact hits only.

In [208]:
gtdb_to_db_proka = pd.read_csv(
    data_folder / 'hydrolase_search' / 'gtdb_mapped_to_db_proka.tsv', 
    sep='\t',
)

gtdb_to_db_proka['coverage'] = gtdb_to_db_proka.apply(
    lambda row: np.round(
        100 * min(row['qlen'], row['tlen']) / max(row['qlen'], row['tlen']), 
        3,
    ), 
    axis=1,
)

gtdb_to_db_proka = gtdb_to_db_proka.sort_values(
    ['query', 'fident', 'mismatch', 'coverage', 'bits'], 
    ascending=[True, False, True, False, False],
).drop_duplicates(
    ['query', 'fident', 'mismatch', 'coverage', 'bits', 'target']
).drop_duplicates('target').set_index('query')

gtdb_to_db_proka = gtdb_to_db_proka[
    (gtdb_to_db_proka['fident'] == 1.0) &
    (gtdb_to_db_proka['mismatch'] == 0) &
    (gtdb_to_db_proka['coverage'] > 99)
]

print(f'Number of matches between GTDB and db_proka: {len(gtdb_to_db_proka):,} of {len(gtdb_hits):,}')

gtdb_to_db_proka.head()

Number of matches between GTDB and db_proka: 191 of 328


,target,qlen,tlen,fident,alnlen,mismatch,qstart,qend,tstart,tend,evalue,bits,coverage
query,,,,,,,,,,,,,
ARWO01000032.1_136,ARWO01000032.1_136@GCA_000402075.1,251,251,1.0,250,0,1,250,1,250,3.563000e-192,603,100.000
BFAR01000126.1_9,GBF24584.1@GCA_008974855.1,261,260,1.0,260,0,1,260,1,260,1.386000e-197,620,99.617
BGOU01000010.1_32,GBL42191.1@GCA_003569705.1,252,251,1.0,251,0,1,251,1,251,3.945000e-190,597,99.603
CACHDH010000007.1_5,CACHDH010000007.1_5@GCA_902578515.1,257,257,1.0,256,0,1,256,1,256,3.822000e-197,618,100.000
CAJWBY010000188.1_8,CAJWBY010000188.1_8@GCA_913020705.1,246,246,1.0,245,0,1,245,1,245,1.233000e-189,595,100.000


In [209]:
gtdb_to_db_proka[gtdb_to_db_proka['target'] == 'WP_256391121.1@GCF_024494695.1']

,target,qlen,tlen,fident,alnlen,mismatch,qstart,qend,tstart,tend,evalue,bits,coverage
query,,,,,,,,,,,,,
NZ_JANHDP010000001.1_573,WP_256391121.1@GCF_024494695.1,280,279,1.0,279,0,1,279,1,279,1.069000e-225,701,99.643


In [210]:
hits_df = gtdb_hits.copy().rename(columns={
    'target': 'gtdb_id',
}).drop(
    columns=['taxlineage'],
)
hits_df['db_proka_id'] = hits_df['gtdb_id'].apply(
    lambda gtdb_id: (
        gtdb_to_db_proka.loc[gtdb_id, 'target'] 
        if gtdb_id in gtdb_to_db_proka.index 
        else None
    )
)
hits_df.head()

,query,gtdb_id,evalue,bits,tstart,tend,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species,db_proka_id
0,pgaptmp_002090_1,NZ_AOLI01000034.1_29,7.391000e-185,581,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax larsenii,WP_007545272.1@GCF_000336955.1
1,pgaptmp_002090_1,NZ_AOLK01000012.1_176,5.303000e-175,552,1,289,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax elongans,None
2,pgaptmp_002090_1,NZ_AOLN01000018.1_554,2.454000e-141,455,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax mucosum,None
3,pgaptmp_002090_1,NC_017941.2_785,3.791000e-139,449,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax mediterranei,WP_004057439.1@GCF_000306765.2
4,pgaptmp_002090_1,NZ_AOLJ01000011.1_106,5.312000e-136,440,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax gibbonsii,WP_004972615.1@GCF_000336775.1


### UniProtKB hits

In [211]:
uniprot_hits = pd.read_csv(data_folder / 'hydrolase_search' / 'uniprot.tsv', sep='\t')

def parse_uniprot_taxonomy(taxon, tax_level):
    for t in taxon.split(';'):
        if t.startswith(f'{tax_level}_'):
            return t.replace(f'{tax_level}_', '')
    return None

uniprot_hits['domain'] = uniprot_hits['taxlineage'].apply(lambda t: parse_uniprot_taxonomy(t, 'd'))
uniprot_hits['ncbi_phylum'] = uniprot_hits['taxlineage'].apply(lambda t: parse_uniprot_taxonomy(t, 'p'))
uniprot_hits['ncbi_class'] = uniprot_hits['taxlineage'].apply(lambda t: parse_uniprot_taxonomy(t, 'c'))
uniprot_hits['ncbi_order'] = uniprot_hits['taxlineage'].apply(lambda t: parse_uniprot_taxonomy(t, 'o'))
uniprot_hits['ncbi_family'] = uniprot_hits['taxlineage'].apply(lambda t: parse_uniprot_taxonomy(t, 'f'))
uniprot_hits['ncbi_genus'] = uniprot_hits['taxlineage'].apply(lambda t: parse_uniprot_taxonomy(t, 'g'))
uniprot_hits['ncbi_species'] = uniprot_hits['taxlineage'].apply(lambda t: parse_uniprot_taxonomy(t, 's'))
uniprot_hits = uniprot_hits[uniprot_hits['domain'].notnull()].set_index('target', drop=True)
uniprot_hits.head()

,query,evalue,bits,tstart,tend,taxlineage,domain,ncbi_phylum,ncbi_class,ncbi_order,ncbi_family,ncbi_genus,ncbi_species
target,,,,,,,,,,,,,
A0A1H7H3G6,pgaptmp_002090_1,5.704000e-188,590,1,290,-_cellular organisms;d_Archaea;p_Euryarchaeota...,Archaea,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax larsenii
M0GTD3,pgaptmp_002090_1,8.030000e-185,581,1,290,-_cellular organisms;d_Archaea;p_Euryarchaeota...,Archaea,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax larsenii
M0HUC4,pgaptmp_002090_1,1.169000e-165,526,1,272,-_cellular organisms;d_Archaea;p_Euryarchaeota...,Archaea,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax elongans
M0I6U3,pgaptmp_002090_1,2.666000e-141,455,1,290,-_cellular organisms;d_Archaea;p_Euryarchaeota...,Archaea,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax mucosum
I3R2Q4,pgaptmp_002090_1,4.120000e-139,449,1,290,-_cellular organisms;d_Archaea;p_Euryarchaeota...,Archaea,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax mediterranei


Distribution of archaeal phyla in UniProtKB:

In [212]:
uniprot_hits[uniprot_hits['domain'] == 'Archaea']['ncbi_phylum'].value_counts(dropna=False)

ncbi_phylum
Nitrososphaerota    169
Euryarchaeota       150
Thermoproteota        1
None                  1
Name: count, dtype: int64

Distribution of bacterial phyla in UniProtKB:

In [213]:
uniprot_hits[uniprot_hits['domain'] == 'Bacteria']['ncbi_phylum'].value_counts(dropna=False)

ncbi_phylum
Gemmatimonadota      3
None                 2
Pseudomonadota       2
Planctomycetota      1
Acidobacteriota      1
Verrucomicrobiota    1
Balneolota           1
Chlamydiota          1
Name: count, dtype: int64

### Map GTDB hits to UniProtKB

In [214]:
uniprot_to_gtdb = pd.read_csv(
    data_folder / 'hydrolase_search' / 'uniprot_mapped_to_gtdb.tsv', 
    sep='\t',
)

uniprot_to_gtdb['coverage'] = uniprot_to_gtdb.apply(
    lambda row: np.round(
        100 * min(row['qlen'], row['tlen']) / max(row['qlen'], row['tlen']), 
        3,
    ), 
    axis=1,
)

uniprot_to_gtdb = uniprot_to_gtdb.sort_values(
    ['query', 'fident', 'mismatch', 'coverage', 'bits'], 
    ascending=[True, False, True, False, False],
).drop_duplicates(
    ['query', 'fident', 'mismatch', 'coverage', 'bits', 'target']
).sort_values(
    ['target', 'coverage', 'bits', 'query'],
    ascending=[True, False, False, True]
).set_index('target').sort_index()

uniprot_to_gtdb = uniprot_to_gtdb[
    (uniprot_to_gtdb['fident'] == 1.0) &
    (uniprot_to_gtdb['mismatch'] == 0) &
    (uniprot_to_gtdb['coverage'] > 99)
]

print(f'Number of matches between GTDB and UniProtKB: {len(uniprot_to_gtdb):,} of {len(uniprot_hits):,}')

uniprot_to_gtdb.head()

Number of matches between GTDB and UniProtKB: 140 of 333


,query,qlen,tlen,fident,alnlen,mismatch,qstart,qend,tstart,tend,evalue,bits,coverage
target,,,,,,,,,,,,,
BFAR01000126.1_9,A0A5J4DVH6,260,261,1.0,260,0,1,260,1,260,2.030000e-197,620,99.617
BGOU01000010.1_32,A0A388QLV9,251,252,1.0,251,0,1,251,1,251,5.780000e-190,597,99.603
BGOU01000010.1_32,A0A4P5YED4,251,252,1.0,251,0,1,251,1,251,5.780000e-190,597,99.603
CM001158.1_702,F3KJK7,185,186,1.0,185,0,1,185,1,185,1.293000e-136,438,99.462
CP046181.1_1955,A0A7D7V958,250,251,1.0,250,0,1,250,1,250,2.462000e-193,607,99.602


Some GTDB proteins match to more than one UnitProtKB protein. In that case we only pick the first one. 

In [215]:
gtdb_id_to_uniprot_id = {
    gtdb_id: uniprot_to_gtdb.loc[[gtdb_id]].iloc[0]['query']
    for gtdb_id in sorted(set(uniprot_to_gtdb.index))
}

hits_df['uniprot_id'] = hits_df['gtdb_id'].apply(
    lambda gtdb_id: gtdb_id_to_uniprot_id.get(gtdb_id)
)
hits_df.head()

,query,gtdb_id,evalue,bits,tstart,tend,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species,db_proka_id,uniprot_id
0,pgaptmp_002090_1,NZ_AOLI01000034.1_29,7.391000e-185,581,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax larsenii,WP_007545272.1@GCF_000336955.1,M0GTD3
1,pgaptmp_002090_1,NZ_AOLK01000012.1_176,5.303000e-175,552,1,289,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax elongans,None,None
2,pgaptmp_002090_1,NZ_AOLN01000018.1_554,2.454000e-141,455,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax mucosum,None,M0I6U3
3,pgaptmp_002090_1,NC_017941.2_785,3.791000e-139,449,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax mediterranei,WP_004057439.1@GCF_000306765.2,I3R2Q4
4,pgaptmp_002090_1,NZ_AOLJ01000011.1_106,5.312000e-136,440,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax gibbonsii,WP_004972615.1@GCF_000336775.1,A0A371N295


Set NCBI taxonomy

In [216]:
for column in ['ncbi_phylum', 'ncbi_class', 'ncbi_order', 'ncbi_family', 'ncbi_genus', 'ncbi_species']:
    hits_df[column] = None
    hits_df[column] = hits_df['uniprot_id'].apply(
        lambda uniprot_id: uniprot_hits.loc[uniprot_id, column] if uniprot_id is not None else None
    )

hits_df.head()

,query,gtdb_id,evalue,bits,tstart,tend,domain,gtdb_phylum,gtdb_class,gtdb_order,...,gtdb_genus,gtdb_species,db_proka_id,uniprot_id,ncbi_phylum,ncbi_class,ncbi_order,ncbi_family,ncbi_genus,ncbi_species
0,pgaptmp_002090_1,NZ_AOLI01000034.1_29,7.391000e-185,581,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,...,Haloferax,Haloferax larsenii,WP_007545272.1@GCF_000336955.1,M0GTD3,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax larsenii
1,pgaptmp_002090_1,NZ_AOLK01000012.1_176,5.303000e-175,552,1,289,Archaea,Halobacteriota,Halobacteria,Halobacteriales,...,Haloferax,Haloferax elongans,None,None,None,None,None,None,None,None
2,pgaptmp_002090_1,NZ_AOLN01000018.1_554,2.454000e-141,455,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,...,Haloferax,Haloferax mucosum,None,M0I6U3,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax mucosum
3,pgaptmp_002090_1,NC_017941.2_785,3.791000e-139,449,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,...,Haloferax,Haloferax mediterranei,WP_004057439.1@GCF_000306765.2,I3R2Q4,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax mediterranei
4,pgaptmp_002090_1,NZ_AOLJ01000011.1_106,5.312000e-136,440,1,290,Archaea,Halobacteriota,Halobacteria,Halobacteriales,...,Haloferax,Haloferax gibbonsii,WP_004972615.1@GCF_000336775.1,A0A371N295,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax sp. Atlit-6N


Add UniProtKB hits not in GTDB

In [217]:
uniprot_hits_not_in_gtdb = uniprot_hits.loc[
    sorted(set(uniprot_hits.index) - set(uniprot_to_gtdb['query'].unique()))
].reset_index().sort_values(
    ['target', 'bits', 'domain', 'ncbi_phylum', 'ncbi_class', 'ncbi_order', 'ncbi_family', 'ncbi_genus', 'ncbi_species'],
    ascending=[True, False] + [True] * 7
).drop_duplicates([
    'bits', 'domain', 'ncbi_phylum', 'ncbi_class', 'ncbi_order', 'ncbi_family', 'ncbi_genus', 'ncbi_species'
])

uniprot_hits_not_in_gtdb = uniprot_hits_not_in_gtdb.drop(
    columns=['taxlineage']
).rename(
    columns={'target': 'uniprot_id'},
)
uniprot_hits_not_in_gtdb['gtdb_id'] = None
uniprot_hits_not_in_gtdb['db_proka_id'] = None

for col in ['gtdb_phylum', 'gtdb_class', 'gtdb_order', 'gtdb_family', 'gtdb_genus', 'gtdb_species']:
    uniprot_hits_not_in_gtdb[col] = None

print('UniProtKB hits not in GTDB:', len(uniprot_hits_not_in_gtdb))

assert len(set(uniprot_hits_not_in_gtdb.columns) & set(hits_df.columns)) == len(uniprot_hits_not_in_gtdb.columns)

UniProtKB hits not in GTDB: 177


In [218]:
output_columns = [
    'gtdb_id', 'db_proka_id', 'uniprot_id',
    'domain', 
    'gtdb_phylum', 'gtdb_class', 'gtdb_order', 'gtdb_family', 'gtdb_genus', 'gtdb_species', 
    'ncbi_phylum', 'ncbi_class', 'ncbi_order', 'ncbi_family', 'ncbi_genus', 'ncbi_species',
    'tstart', 'tend', 'evalue', 'bits',
]
search_output_df = pd.concat(
    [
        hits_df[output_columns],
        uniprot_hits_not_in_gtdb[output_columns],
    ],
    ignore_index=True,
).sort_values(
    'bits',
    ascending=False
).reset_index(drop=True)
search_output_df.head()

,gtdb_id,db_proka_id,uniprot_id,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species,ncbi_phylum,ncbi_class,ncbi_order,ncbi_family,ncbi_genus,ncbi_species,tstart,tend,evalue,bits
0,None,None,A0A1H7H3G6,Archaea,None,None,None,None,None,None,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax larsenii,1,290,5.704000e-188,590
1,NZ_AOLI01000034.1_29,WP_007545272.1@GCF_000336955.1,M0GTD3,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax larsenii,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax larsenii,1,290,7.391000e-185,581
2,NZ_AOLK01000012.1_176,None,None,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax elongans,None,None,None,None,None,None,1,289,5.303000e-175,552
3,None,None,M0HUC4,Archaea,None,None,None,None,None,None,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax elongans,1,272,1.169000e-165,526
4,NZ_AOLN01000018.1_554,None,M0I6U3,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax mucosum,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax mucosum,1,290,2.454000e-141,455


### Make unique id

In this order:
- uniprot ID if available
- db_proka ID if available
- GTDB ID otherwise


In [219]:
def set_id(row):
    if not pd.isnull(row['uniprot_id']):
        return row['uniprot_id']
    elif not pd.isnull(row['db_proka_id']):
        return row['db_proka_id']
    elif pd.isnull(row['gtdb_id']):
        raise ValueError('All IDs are null')
    else:
        return row['gtdb_id']


search_output_df['id'] = search_output_df.apply(set_id, axis=1)
search_output_df = search_output_df.set_index('id')
search_output_df.head()

,gtdb_id,db_proka_id,uniprot_id,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species,ncbi_phylum,ncbi_class,ncbi_order,ncbi_family,ncbi_genus,ncbi_species,tstart,tend,evalue,bits
id,,,,,,,,,,,,,,,,,,,,
A0A1H7H3G6,None,None,A0A1H7H3G6,Archaea,None,None,None,None,None,None,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax larsenii,1,290,5.704000e-188,590
M0GTD3,NZ_AOLI01000034.1_29,WP_007545272.1@GCF_000336955.1,M0GTD3,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax larsenii,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax larsenii,1,290,7.391000e-185,581
NZ_AOLK01000012.1_176,NZ_AOLK01000012.1_176,None,None,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax elongans,None,None,None,None,None,None,1,289,5.303000e-175,552
M0HUC4,None,None,M0HUC4,Archaea,None,None,None,None,None,None,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax elongans,1,272,1.169000e-165,526
M0I6U3,NZ_AOLN01000018.1_554,None,M0I6U3,Archaea,Halobacteriota,Halobacteria,Halobacteriales,Haloferacaceae,Haloferax,Haloferax mucosum,Euryarchaeota,Halobacteria,Haloferacales,Haloferacaceae,Haloferax,Haloferax mucosum,1,290,2.454000e-141,455


In [220]:
search_output_df.to_csv(data_folder / 'hydrolase_search' / 'search_output.csv')

In [221]:
search_output_df['domain'].value_counts()

domain
Archaea     480
Bacteria     25
Name: count, dtype: int64

### Export fasta

In [222]:
def taxomomy_str(row, source):
    taxonomy_ranks = [
        ('domain', 'd'),
        (f'{source}_phylum', 'p'),
        (f'{source}_class', 'c'),
        (f'{source}_order', 'o'),
        (f'{source}_family', 'f'),
        (f'{source}_genus', 'g'),
        (f'{source}_species', 's'),
    ]
    tax_str = ';'.join([f'{level}_{row[c]}' for c, level in taxonomy_ranks if not pd.isnull(row[c])])
    return f'{source}_taxonomy={tax_str}'

In [223]:
records = []
for record in SeqIO.parse(data_folder / 'hydrolase_2090.fasta', 'fasta'):
    record.id = 'pgaptmp_002090_1'
    records.append(record)

count = 0
gtdb_ids = set(search_output_df['gtdb_id'].unique())
seen_ids = set()
for record in SeqIO.parse(data_folder / 'hydrolase_search' / 'gtdb.fasta', 'fasta'):
    if record.id in gtdb_ids:
        row = search_output_df[search_output_df['gtdb_id'] == record.id].iloc[0]
        record.name = ''
        record.description = taxomomy_str(row, 'gtdb')
        new_id = row.name
        record.id = new_id
        if record.seq.endswith("*"):
            record.seq = record.seq[:-1]
        records.append(record)
        seen_ids.add(new_id)
        count += 1

uniprot_ids = set(search_output_df['uniprot_id'].unique())
for record in SeqIO.parse(data_folder / 'hydrolase_search' / 'uniprot.fasta', 'fasta'):
    record_id = record.id.split('|')[1]
    if record_id in uniprot_ids:
        row = search_output_df[search_output_df['uniprot_id'] == record_id].iloc[0]
        new_id = row.name
        if new_id not in seen_ids:
            record.id = new_id
            record.name = ''
            record.description = taxomomy_str(row, 'ncbi')
            if record.seq.endswith("*"):
                record.seq = record.seq[:-1]
            records.append(record)
            seen_ids.add(new_id)
            count += 1

assert count == len(search_output_df), (count, len(search_output_df))

with (data_folder / 'hydrolase_search' / 'search_output.fasta').open('w') as f_out:
    SeqIO.write(records, f_out, 'fasta')

### Process hmmer Pfam search

`hmmer` Pfam (release 37) search from script [`src/homology_search/predict_pfam.sh`](src/homology_search/predict_pfam.sh).

In [229]:
def process_hmmer_output(domtblout_path):
    output_data = {
        'protein_id': [],
        'hmm_accession': [],
        'hmm_query': [],
        'evalue': [],
        'bitscore': [],
        'accuracy': [],
        'start': [],
        'end': [],
    }
    with domtblout_path.open() as f:
        parser = Hmmer3DomtabHmmqueryParser(f)

        for record in parser:
            hmm_accession = record.accession
            for protein_id, hit in record.items:
                hmm_query = hit.query_id
                for hit_instance in hit:
                    evalue = hit_instance.evalue
                    bitscore = hit_instance.bitscore
                    start = hit_instance.env_start
                    end = hit_instance.env_end
                    accuracy = hit_instance.acc_avg

                    accession = hmm_accession
                    if accession == '-' or accession is None:
                        accession = hmm_query

                    output_data['protein_id'].append(protein_id)
                    output_data['hmm_accession'].append(accession)
                    output_data['hmm_query'].append(hmm_query)
                    output_data['evalue'].append(evalue)
                    output_data['bitscore'].append(bitscore)
                    output_data['accuracy'].append(accuracy)
                    output_data['start'].append(start)
                    output_data['end'].append(end)

    out_df = pd.DataFrame.from_dict(output_data).sort_values(['protein_id', 'start']).set_index('protein_id')

    return process_overlapping_domains(out_df)


def process_overlapping_domains(domains_df):
    """
    If two domains are overlapping within the same protein, keep longest only.
    """
    dfs = []
    for protein_id in sorted(set(domains_df.index)):
        df = domains_df.loc[[protein_id]].reset_index()

        overlapping_id_pairs = find_overlapping_entries(df)

        blacklist = set()
        for id_1, id_2 in overlapping_id_pairs:
            if id_1 in blacklist or id_2 in blacklist:
                continue

            row_1 = df.loc[id_1]
            row_2 = df.loc[id_2]
            
            l1 = row_1['end'] - row_1['start']
            l2 = row_2['end'] - row_2['start']

            if l2 > l1:
                blacklist.add(id_1)
            else:
                blacklist.add(id_2)

        final_set = set(df.index) - blacklist
        dfs.append(
            df.loc[sorted(final_set)].reset_index(drop=True)
        )

    return pd.concat(
        dfs, 
        ignore_index=True,
    ).sort_values(
        ['protein_id', 'start']
    ).set_index(
        'protein_id',
        drop=True,
    )


def find_overlapping_entries(df):
    tree = IntervalTree()

    # Populate the tree with intervals from the DataFrame
    for idx, row in df.iterrows():
        tree[row['start']:row['end']+1] = idx

    # Find overlapping intervals
    overlaps = set()
    for interval in tree:
        overlapping = tree.overlap(interval.begin, interval.end)
        if len(overlapping) > 1:
            for overlapped_interval in overlapping:
                if overlapped_interval.data != interval.data:
                    overlaps.add(frozenset([interval.data, overlapped_interval.data]))

    # Process the overlaps to make them more readable
    return [tuple(pair) for pair in overlaps]


pfam_df = process_hmmer_output(data_folder / 'hydrolase_search' / 'search_output.pfam.domtblout.txt')
pfam_df.to_csv(data_folder / 'hydrolase_search' / 'search_output.pfam.csv')
pfam_df.head()

,hmm_accession,hmm_query,evalue,bitscore,accuracy,start,end
protein_id,,,,,,,
A0A075FKQ7,PF05990.17,DUF900,4.900000e-10,34.2,0.80,31,220
A0A075GAR9,PF05990.17,DUF900,3.600000e-08,28.1,0.82,39,210
A0A075GDE7,PF05990.17,DUF900,3.800000e-11,37.8,0.83,37,215
A0A075GE09,PF05990.17,DUF900,3.800000e-08,28.0,0.78,31,206
A0A075GFP7,PF05990.17,DUF900,3.600000e-08,28.1,0.82,39,210


In [225]:
top_pfam_domains = pfam_df.reset_index()[
    ['hmm_accession', 'hmm_query', 'protein_id']
].groupby(['hmm_accession', 'hmm_query']).nunique().rename(columns={
    'protein_id': 'n_proteins',
}).sort_values('n_proteins', ascending=False)
top_pfam_domains['percent'] = (100 * top_pfam_domains['n_proteins'] / (count + 1)).round(1)
top_pfam_domains

,,n_proteins,percent
hmm_accession,hmm_query,,
PF05990.17,DUF900,349,69.0
PF05277.17,DUF726,69,13.6
PF01083.27,Cutinase,5,1.0
PF00652.27,Ricin_B_lectin,3,0.6
PF05057.19,DUF676,2,0.4
PF06057.16,VirJ,2,0.4
PF18911.5,PKD_4,2,0.4
PF01471.23,PG_binding_1,1,0.2
PF01510.30,Amidase_2,1,0.2


In [226]:
architectures_df = pfam_df.reset_index()[['protein_id']].drop_duplicates('protein_id').set_index('protein_id')
architectures_df['pfam_architecture'] = None

for protein_id in sorted(set(architectures_df.index)):
    domains = pfam_df.loc[[protein_id]]['hmm_query'].values
    architectures_df.loc[protein_id, 'pfam_architecture'] = '; '.join(domains)

architectures_df['domain'] = [
    search_output_df.loc[protein_id, 'domain'] if protein_id != 'pgaptmp_002090_1' else 'Archaea'
    for protein_id in architectures_df.index
]

architectures_df.head()

,pfam_architecture,domain
protein_id,,
A0A075FKQ7,DUF900,Archaea
A0A075GAR9,DUF900,Archaea
A0A075GDE7,DUF900,Archaea
A0A075GE09,DUF900,Archaea
A0A075GFP7,DUF900,Archaea


In [228]:
architectures_df.reset_index().groupby(['domain', 'pfam_architecture']).nunique().rename(columns={
    'protein_id': 'n_proteins',
}).sort_values('n_proteins', ascending=False)

n_proteins
domain   pfam_architecture                             
Archaea  DUF900                                     327
         DUF726                                      64
Bacteria DUF900                                      18
Archaea  Cutinase                                     5
Bacteria DUF726                                       5
Archaea  Ricin_B_lectin                               3
         DUF676                                       2
         VirJ                                         2
Bacteria PKD_4; DUF900                                2
Archaea  Abhydrolase_6                                1
         Abhydrolase_8                                1
         DUF900; Amidase_2                            1
         DUF900; PG_binding_1; PG_binding_1           1